In [3]:
%%bash
pip install pint 


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3 -m pip install --upgrade pip' command.


In [47]:
import pint
import primemover
from pint import UnitRegistry
ureg=UnitRegistry()


In [75]:
#_________________________________________TREATMENT_DESIGN_INPUTS_________________________________________________________________________

#GENERAL FRAC INPUTS - PRIME MOVERS| PUMPS BELNDERS ETC
frac_type="conv"# [conv, zip, sim]
num_stage = 50 #[30, 150]
stage_time = 120 #[60,240]min
prime_mover_fuel="dis" #[dis, cng, lng, elect]	
fuel_fraction_dis=100 #[0,100]	per
fuel_fraction_cng=0  #[0,100]	per
fuel_fraction_lng=0 #[0,100]	per
fuel_fraction_elect=0 #[0,100]	per
TBS=120 #[60, 240]	min

#TREATEMENT DATA IMNPUTS
P=9000 #bottomohole pressure in psi
Q=90 #Rate of pumping barrels per minute#Horsepower
etta=0.9 # pump efficiency 
HHP=(P*Q)/(40.8*etta)
print(HHP)
numberofpumps=HHP/2250 #Number of pumps estimate

#SAND DESIGN 
water = 430000 # gal [100000, 1000000]
cfr=2.5  #kg/gal

#SAND DESIGN 
prop_per_stage=20 # range [10, 1000]	ton
prop_den =3.0 #[2,4]	ton/m3
prop_capacity = 1000 #[0, 2000]	ton
prop_sys_used="silo" #[silo, box, bunker]	
prop_type="sand" #[sand, ceramics, resine, bauaxite]	
prop_footprint=0.24 # [0, 0.5]	tonCO2/ton_propant has to be replaced or assumed by contractor


# TRANSPORTATION FUEL CALCULATION 
transp_fuel="dis" #[dis, cng, lng, gasoline, elect]
t_fuel_fraction_dis=100 #[0,100]	per
t_fuel_fraction_cng=0  #[0,100]	per
t_fuel_fraction_lng=0 #[0,100]	per
t_fuel_fraction_elect=0 #[0,100]	per
dist_to_water= 70 #	[0, 200]	miles # used in case of trucking the water 
dist_to_sand= 70  #	[0, 200]	miles # used in case of trucking the sand
dist_to_fuel= 70  #	[0, 200]	miles # used in case of trucking the sand

#EMISSSION CONSTANTS

# Fuel constants
CO2=10.21 #kg/gal 
CH4E=0.2/1000*25 #kg/gal
N2OE=0.47/1000*298 #kg/gal

heavy_truck_consumption_dis = 10 # 5-20 mpg user input based on fleet year
med_truck_consumption=14 # 10-25 mpg

Diesel_LHVbtupergal = 137381 #temporary substituting with the source 
#https://www.eia.gov/energyexplained/units-and-calculators/british-thermal-units.php
#Horesepower/hr to BTU - conversion factor ||1 horsepower (hp) = 2,544.43 btu per hour (btu/h)||
BTUhr=HHP*2544.43
print(BTUhr)
#CO2 Factors 
#Diesel conversion. factor CO2 =10.21 kg/gal, CH4 0.2 g/gal, 0.47 g - gal - source EPA table. https://www.epa.gov/sites/production/files/2020-04/documents/ghg-emission-factors-hub.pdf
#CO2e conversion CH4 - 25. N20 - 298
#Diesel LHV 12425 btu/lb; Diesel density 3.202 kg/gal; https://www.engineeringtoolbox.com/fuels-higher-calorific-values-d_169.html


22058.823529411766
56127132.35294118


In [77]:
#_________________________________PRIME______________________________MOVER____________________________________________

V_fuel_hr=BTUhr/Diesel_LHVbtupergal# Fuel consumption per hour
V_fuel_hr=V_fuel_hr
print(V_fuel_hr)

#Convert to CO2e
pm_CO2=V_fuel_hr*CO2
pm_CH4E=V_fuel_hr*CH4E
pm_N2OE=V_fuel_hr*N2OE
pm_CO2E=pm_CO2+pm_CH4E+pm_N2OE
print(pm_CO2E) # CO2e in kg

total_pm_CO2=pm_CO2*stage_time/60*num_stage #assuming pumps are not running in TBS
print (total_pm_CO2)
total_pm_CO2*ureg.kg
print (total_pm_CO2/1000)

408.55090844397097
4230.569169991827
417130.47752129444
417.13047752129444


In [78]:
#Transportation classes: Sand, Water, Fuel, Mob-Demob, 
sand_load=24 # sand load in metric tonns
water_load=20 # M3 of water load in case of land by truck transportation
fuel_load=10 # M3 in case of of land by track transportation and immediate distribution

#Mob Demob
t_pumps = 18 # Input that needs to match with first module
t_tanks = 25 # number of trucks to travel to location
t_blender=1
t_pcm=1
t_silos=4
t_iron =3
t_missle =2
t_LAS = 2
t_crane =1
t_wlu=1
t_commandunit=1
t_pickup=10
t_aux=20

mob_distance= 80 #[1,500] generalized variable, 
#It is very well might be that the mobilization units will be different for each of the pieces of the equipment

# TRANSPORTATION FUEL CALCULATION 
#Mobilization - DIESEL simplified model 

Mob_volFuel=(t_pumps+t_tanks+t_blender+t_pcm+t_silos+t_iron+t_missle+t_LAS+t_crane+t_wlu+t_commandunit)*mob_distance/heavy_truck_consumption_dis
Mob_volFuel=Mob_volFuel+(t_pickup+t_aux)*mob_distance/med_truck_consumption
#Simplified model includes all pieces of the equipment as heavy trucks from the mobilization purposes, 
#pickups and auxilarry equipment - all as diesel and medium trucks. 

Mob_CO2e=Mob_volFuel*CO2+Mob_volFuel*CH4E+Mob_volFuel*N2OE # Estimate of KG CO2e 

print (Mob_CO2e)



6662.741462857143


In [80]:
# Sand total emissions for transportation assuming all the trucks are doing transportation on diesel
sand_number_of_loads=prop_per_stage*num_stage/sand_load
print (sand_number_of_loads)
sand_fuel=sand_number_of_loads*dist_to_sand/med_truck_consumption # assumption of a average truck consumption 14 mpg
sand_CO2e=sand_fuel*CO2+sand_fuel*CH4E+sand_fuel*N2OE
print(sand_CO2e)

41.666666666666664
2157.304166666667


In [83]:
# Water total emissions for transportation assuming all the trucks are doing transportation on diesel
# note only valid if water not locally sourced from in a pit.
water_number_of_loads=water*0.00378541*num_stage/water_load # coefficient is to transfer 
print(water_number_of_loads)
water_fuel=water_number_of_loads*dist_to_sand/med_truck_consumption # assumption of a average truck consumption 14 mpg
water_CO2e=water_fuel*CO2+water_fuel*CH4E+water_fuel*N2OE
water_CO2e=water_CO2e
print(water_CO2e)


4069.31575
210690.04375097508


In [65]:
# Fuel transport missions for transportation assuming all the trucks are doing transportation on diesel

fuel_number_of_loads=V_fuel_hr.to(ureg.m**3)*stage_time/60*num_stage /fuel_load
print(fuel_number_of_loads)
fuel_fuel=fuel_number_of_loads*dist_to_sand/med_truck_consumption # assumption of a average truck consumption 14 mpg
fuel_CO2e=fuel_fuel*CO2+fuel_fuel*CH4E+fuel_fuel*N2OE
fuel_CO2e=fuel_CO2e
print(fuel_CO2e)



15.465334231877128 meter ** 3
800.7223194557079 meter ** 3


In [66]:
TotalCO2e=total_pm_CO2+water_CO2e+sand_CO2e+Mob_CO2e+fuel_CO2e
print(TotalCO2e)

ValueError: Cannot operate with Quantity and Quantity of different registries.